# Train and Apply Models

In [1]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest,
    build_lstm_sequences
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [2]:
# remove_list = [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 14, 16, 17, 18, 19, 22]
remove_list = []

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

# subjects = [3, 12, 15]
# subjects = []
# for _ in range(5):
#     subjects.append(random.randint(0, 22))
# print(subjects)
subjects = [i for i in range(0, 23)]

## LSTM CV Optimizer

In [10]:
param_grid = {
    "lr": [0.0001],
    "epochs": [100],
    "units": [512],
    "batch_size": [128],
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train_df, X_test_df, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table.csv",
                    relabel=relabel,
                )

                features = utils.filter_features(
                    X_train_df.columns,
                    remove_bands=["gamma", "delta"],
                )
                features = [c for c in features if c in X_train_df.columns]

                # Build sequence-level data (trials = (patient, video))
                X_train_seq, y_train_seq = build_lstm_sequences(
                    X_train_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    # fixed_T=15,
                )
                X_test_seq, y_test_seq = build_lstm_sequences(
                    X_test_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    # fixed_T=15
                )

                minority = (y_train_seq == 1.0).sum() < (y_train_seq == 0.0).sum()
                maj_label = 0.0 if minority else 1.0
                min_label = 1.0 - maj_label

                idx_maj = np.where(y_train_seq == maj_label)[0]
                idx_min = np.where(y_train_seq == min_label)[0]

                if len(idx_min) == 0:
                    # no samples of one class, redo split
                    print("No minority class in this split, re-drawing...")
                    continue

                # reps = int(np.ceil(len(idx_maj) / len(idx_min)))
                # idx_min_upsampled = np.tile(idx_min, reps)[: len(idx_maj)]

                # idx_balanced = np.concatenate([idx_maj, idx_min_upsampled])
                # np.random.shuffle(idx_balanced)

                # X_train_bal = X_train_seq[idx_balanced]
                # y_train_bal = y_train_seq[idx_balanced]

                print("y_train counts:", np.bincount(y_train_seq.astype(int)))
                print("y_test counts:", np.bincount(y_test_seq.astype(int)))

                break

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_seq,
                X_test_seq,
                y_train_seq,
                y_test_seq,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.4,
                recurrent_dropout=0.2,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            if acc < 0.5:
                print("Rerun due to inverted labeling:", acc)
                arousal_pred = (y_prob <= 0.5).astype(int)
                acc = accuracy_score(y_test_eval, arousal_pred)
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
        }


print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 0 | Held-out (patient, video) trials: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17)] | Excluded users: []
y_train counts: [220 176]
y_test counts: [13  5]
X_train_arr shape: (396, 196, 452)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step
0.6666666666666666

Confusion Matrix (subject):
[[11  2]
 [ 4  1]]
Held-out patient: 1 | Held-out (patient, video) trials: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17)] | Excluded users: []
y_train counts: [224 172]
y_test counts: [9 9]
X_train_arr shape: (396, 196, 452)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 691ms/step
0.6666666666666666

Confusion Matrix (subject):
[[9 0]
 [6 3]]
Held-out patient: 2 | Held-out (patient, video) trials: [(2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2,

KeyboardInterrupt: 

### Fine-tune for User

In [ ]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

acc_list = []

# Must reuse the SAME feature_cols and fixed_T as in global training
# feature_cols and fixed_T assumed defined above

for run in range(0, 18):
    X_user_train_df, X_user_test_df, arousal_user_train, arousal_user_test = (
        single_user_split(
            target="arousal",
            selected_user=21,
            k_holdouts=1,
            filename="datasets/features_table.csv",
        )
    )
    X_user_train_seq, y_user_train_seq = build_lstm_sequences(
        X_user_train_df,
        feature_cols=features,
        target_col="arousal",
        thresh=3.8,
    )
    X_user_test_seq, y_user_test_seq = build_lstm_sequences(
        X_user_test_df,
        feature_cols=features,
        target_col="arousal",
        thresh=3.8,
    )

    print(f"Run {run}")
    print("User train seq shape:", X_user_train_seq.shape)
    print("User test  seq shape:", X_user_test_seq.shape)
    print("y_train counts:", np.bincount(y_user_train_seq.astype(int)))
    print("y_test  counts:", np.bincount(y_user_test_seq.astype(int)))

    user_model = clone_model(best_lstm)
    user_model.build(best_lstm.input_shape)
    user_model.set_weights(best_lstm.get_weights())

    for layer in user_model.layers[:-1]:
        layer.trainable = False

    user_model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history = user_model.fit(
        X_user_train_seq,
        y_user_train_seq,
        validation_split=0.2,
        verbose=False,
        batch_size=64,
        epochs=10,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=3,
                restore_best_weights=True,
                monitor="val_loss",
            )
        ],
    )

    test_loss, test_acc = user_model.evaluate(
        X_user_test_seq, y_user_test_seq, verbose=0
    )
    print(f"\nRun {run} - User Test accuracy: {test_acc:.4f}")

    acc_list.append(test_acc)

    # Predictions & confusion matrix
    y_prob = user_model.predict(X_user_test_seq, verbose=0).ravel()
    y_pred = (y_prob >= 0.5).astype("int32")

    cm = confusion_matrix(y_user_test_seq.astype("int32"), y_pred, labels=[0, 1])
    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(cm)

    print(
        "\nOverall accuracy:", accuracy_score(y_user_test_seq.astype("int32"), y_pred)
    )

acc_array = np.array(acc_list)
print(f"\nMean user test accuracy over {len(acc_array)} runs: {acc_array.mean():.4f}")
print(f"Std of user test accuracy over {len(acc_array)} runs: {acc_array.std():.4f}")

Run 0
User train seq shape: (17, 196, 452)
User test  seq shape: (1, 47, 452)
y_train counts: [ 7 10]
y_test  counts: [1]

Run 0 - User Test accuracy: 0.0000


ValueError: Input 0 of layer "functional_22" is incompatible with the layer: expected shape=(None, 196, 452), found shape=(1, 47, 452)